In [3]:
##!pip install google-api-python-client


In [5]:
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime


In [ ]:
youTubeApiKey = "youTubeApiKey"

In [ ]:
youtube=build('youtube','v3', developerKey=youTubeApiKey)

In [ ]:
# Search for the channel using the custom handle
channel_handle = '@cuiabaesporteclube1572'
search_response = youtube.search().list(
    q=channel_handle,
    type='channel',
    part='id,snippet',
    maxResults=1
).execute()

In [ ]:
if 'items' in search_response and len(search_response['items']) > 0:
    channel_id = search_response['items'][0]['snippet']['channelId']
    print(f"Channel ID: {channel_id}")

    try:
        # Get the uploads playlist ID for the channel
        res = youtube.channels().list(part='contentDetails', id=channel_id).execute()
        if 'items' in res and len(res['items']) > 0:
            uploads_playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        else:
            print("Error: Channel not found or no content details available.")
            uploads_playlist_id = None
    except Exception as e:
        print(f"An error occurred: {e}")
        uploads_playlist_id = None

    if uploads_playlist_id:
        # Initialize variables
        playlist_videos = []
        nextPage_token = None


Channel ID: UCnvMjCptspOne0ztN7z-KHg


In [ ]:
# Function to get uploads playlist ID
def get_uploads_playlist_id(channel_id):
    res = youtube.channels().list(part='contentDetails', id=channel_id).execute()
    if 'items' in res and len(res['items']) > 0:
        return res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    else:
        raise ValueError("No content details available for the channel.")

In [ ]:
# Function to get videos from the playlist
def get_playlist_videos(playlist_id, max_results=50):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=min(max_results - len(videos), 50),
            pageToken=next_page_token
        ).execute()

        videos.extend(res['items'])
        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break

    return videos

In [ ]:
# Extract video IDs
videos_ids = list(map(lambda x: x['snippet']['resourceId']['videoId'], playlist_videos))

In [ ]:
# Function to extract video statistics
def get_video_stats(video_ids):
    stats = []
    for video_id in video_ids:
        res = youtube.videos().list(part='statistics', id=video_id).execute()
        stats.extend(res['items'])
    return stats

In [ ]:
# Main script
channel_handle = '@atletico'
uploads_playlist_id = get_uploads_playlist_id(channel_id)
playlist_videos = get_playlist_videos(uploads_playlist_id, max_results=50)

In [ ]:
# Extract video details
videos_ids = [video['snippet']['resourceId']['videoId'] for video in playlist_videos]
stats = get_video_stats(videos_ids)

In [ ]:
videos_title = list(map(lambda x: x['snippet']['title'], playlist_videos))
url_thumbnails = list(map(lambda x: x['snippet']['thumbnails'], playlist_videos))
published_date = list(map(lambda x: str(x['snippet']['publishedAt']), playlist_videos)) #conversion from ISO8601 date format
video_description = list(map(lambda x: x['snippet']['description'], playlist_videos))
videoid = list(map(lambda x: x['snippet']['resourceId']['videoId'], playlist_videos))


In [ ]:
liked = list(map(lambda x: int(x['statistics']['likeCount']), stats))
views = list(map(lambda x: int(x['statistics']['viewCount']), stats))
comment = list(map(lambda x: int(x['statistics']['commentCount']), stats))

In [ ]:

extraction_date = [str(datetime.now())]*len(videos_ids)

In [ ]:
playlist_df = pd.DataFrame({'title':videos_title,
      'video_id':videoid,
      'video_description':video_description,
      'published_date':published_date,
      'extraction_date':extraction_date,
      'likes':liked,
      'views':views,
      'comment':comment,
      'thumbnail': url_thumbnails})
playlist_df.head()

,title,video_id,video_description,published_date,extraction_date,likes,views,comment,thumbnail
0,Elenco do Dourado encerra a preparação para en...,XWqoD3Jl6Ps,São Paulo x Cuiabá se enfrentam nesta quarta-f...,2024-06-18T15:39:55Z,2024-06-18 18:28:15.507540,37,136,2,{'default': {'url': 'https://i.ytimg.com/vi/XW...
1,Os bastidores da goleada por 5 a 0 diante do F...,CoFsgGqD-hY,Dourado cumpriu o dever de casa com placar elá...,2024-06-17T21:51:53Z,2024-06-18 18:28:15.507540,132,1125,16,{'default': {'url': 'https://i.ytimg.com/vi/Co...
2,"Artilheiro do ano, o atacante Isidro Pitta ava...",VhwRSTpgFdw,O paraguaio já marcou 17 gols na temporada 2024,2024-06-17T19:43:09Z,2024-06-18 18:28:15.507540,123,972,17,{'default': {'url': 'https://i.ytimg.com/vi/Vh...
3,Técnico Petit analisa a vitória diante do Fort...,iJatJkDUglU,Comandante do Dourado falou sobre a goleada co...,2024-06-17T01:44:22Z,2024-06-18 18:28:15.507540,199,2628,32,{'default': {'url': 'https://i.ytimg.com/vi/iJ...
4,Confira os gols da goleada do Dourado por 5 a ...,4jA4FrnaxFo,Cuiabá mostrou sua força e conquistou a primei...,2024-06-17T00:18:50Z,2024-06-18 18:28:15.507540,153,1557,17,{'default': {'url': 'https://i.ytimg.com/vi/4j...


In [ ]:
playlist_df.to_csv('cuiabatv.csv', index=False)
